<a href="https://colab.research.google.com/github/MustafaEisaTech/Learning_TF/blob/main/Custom_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Layer

In [4]:
class CustomDenseLayer(keras.layers.Layer):
     def __init__(self, units=32):
        super(CustomDenseLayer, self).__init__()
        self.units = units
     def build(self, input_shape):
        self.w = self.add_weight(
            shape = (input_shape[-1], self.units),
            Initializer = "random_normal",
            trainable = True
        )
        self.b = self.add_weight(
            shape = (self.units, ),
            Initializer = "zeros",
            trainable = True
        )

     def call(self, inputs):
      return tf.matmul(inputs, self.w) + self.b

In [5]:
class BadDenseLayer(keras.layers.Layer):
    def __init__(self, units = 32) -> None:
       super(BadDenseLayer, self).__init__()
       self.units = units
    def build(self):
      self.w = self.add_weight(
          shape = (64, self.units),
          Initializer = "random_normal",
          trainable = True
      )
      self.b = self.add_weight(
          shape = (self.units, ),
          initializer = "zeros",
          trainable = True
      )
    def call(self, inputs):
      return tf.matmul(inputs, self.w) + self.b


In [6]:
class ForgetGate(keras.layers.Layer):
    def __init__(self, units) -> None:
        super(ForgetGate, self).__init__()

    def build(self, input_shape):
        input_dim = input_shape[0][-1]
        hidden_dim = input_shape[1][-1]
        cell_dim = input_shape[2][-1]

        self.w = self.add_weight(
            shape = (input_dim + hidden_dim + cell_dim, self.units),
            initializer = "random_normal",
            trainable = True
        )
        self.b = self.add_weight(
            shape = (self.units, ),
            initializer = "zeros",
            trainable = True,
            name = "Forget gate bias"
        )
    def call(self, inputs):
      x, h_prev, c_prev = inputs

      concatinated_input = tf.concat([x, h_prev, c_prev], axis = -1)

      forget_gate = tf.sigmoid(tf.matmul(concatinated_input, self.w) + self.b)

      return forget_gate